# Big Data in Finance II  Group Assignment

#### Question1.	In the data used by Gu, Kelly and Xiu (RFS 2019 – provided in class), use a similar procedure to theirs to predict stock returns with neural networks. Start by finding a suitable baseline configuration, and use a validation procedure to pick optimal hyperparameters for three neural network models: One with 2 hidden layers, one with 3 hidden layers, and one with 4 hidden layers.

Import the packages and data

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
import optuna

# fix random state
random_state = 42

panel = pd.read_pickle('returns_chars_panel.pkl') 
macro = pd.read_pickle('macro_timeseries.pkl')

ImportError: Traceback (most recent call last):
  File "C:\Users\56909\PycharmProjects\pythonProject\venv\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 70, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: 页面文件太小，无法完成操作。


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

Process the data

In [26]:
# Assuming 'excess_ret' is the target variable
X = panel.drop(columns=['date', 'permno', 'excess_ret', 'ret']).values
y = panel['excess_ret'].values

# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


MemoryError: Unable to allocate 2.62 GiB for an array with shape (94, 3739449) and data type float64

In [23]:
X_train.shape

(608092, 105)

Train the NN with optuna

In [14]:

def create_model(trial, num_layers):
    neurons_per_layer = trial.suggest_categorical('neurons_per_layer', [32, 64, 128, 256])
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd'])
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])

    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))
    model.add(Dense(neurons_per_layer, activation=activation))

    for _ in range(num_layers - 1):
        model.add(Dense(neurons_per_layer, activation=activation))
        model.add(BatchNormalization())
        model.add(Dropout(0.2))

    model.add(Dense(1))  # Output layer for regression

    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        opt = SGD(learning_rate=learning_rate)

    model.compile(optimizer=opt, loss='mean_squared_error', metrics=['mae'])
    return model, batch_size

def objective(trial, num_layers):
    model, batch_size = create_model(trial, num_layers)

    # K-Fold Cross Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    val_scores = []

    for train_index, val_index in kf.split(X_train):
        X_tr, X_val = X_train[train_index], X_train[val_index]
        y_tr, y_val = y_train[train_index], y_train[val_index]

        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=10)
        model.fit(X_tr, y_tr, epochs=100, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=[es], verbose=0)

        val_loss, val_mae = model.evaluate(X_val, y_val, verbose=0)
        val_scores.append(val_mae)

    return np.mean(val_scores)

# Create a study for each number of layers and optimize
studies = {}
num_layers_options = [2, 3, 4]

for num_layers in num_layers_options:
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, num_layers), n_trials=50)
    studies[num_layers] = study
    print(f'Best hyperparameters for {num_layers} layers: {study.best_params}')
    print(f'Validation MAE for {num_layers} layers: {study.best_value}')

# Evaluate the best model on the test set
for num_layers, study in studies.items():
    best_params = study.best_params
    neurons_per_layer = best_params['neurons_per_layer']
    activation = best_params['activation']
    optimizer = best_params['optimizer']
    learning_rate = best_params['learning_rate']
    batch_size = best_params['batch_size']

    final_model, _ = create_model(study.best_trial, num_layers)

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    final_model.fit(X_train, y_train, epochs=100, batch_size=batch_size, validation_split=0.2, callbacks=[es], verbose=1)

    test_loss, test_mae = final_model.evaluate(X_test, y_test)
    print(f'Test MAE for {num_layers} layers: {test_mae}')

[I 2024-05-16 17:42:04,178] A new study created in memory with name: no-name-ee6eb7b5-cd12-4308-bb3d-b7a3a72dac40
[W 2024-05-16 17:53:46,875] Trial 0 failed with parameters: {'neurons_per_layer': 128, 'activation': 'tanh', 'optimizer': 'sgd', 'learning_rate': 0.017296233402583173, 'batch_size': 32} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\56909\PycharmProjects\pythonProject\venv\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\56909\AppData\Local\Temp\ipykernel_2716\1032699209.py", line 51, in <lambda>
    study.optimize(lambda trial: objective(trial, num_layers), n_trials=50)
  File "C:\Users\56909\AppData\Local\Temp\ipykernel_2716\1032699209.py", line 38, in objective
    model.fit(X_tr, y_tr, epochs=100, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=[es], verbose=0)
  File "C:\Users\56909\PycharmProjects\pythonProject\venv\lib\sit

KeyboardInterrupt: 

Write the analysis here.

#### Question2.	Use test data to get an idea of the out of sample performance of each model. Convert the standard MSE metric for out of sample performance to the “R2 out of sample” metric that was discussed in class. Compare your results to those in Gu-Kelly-Xiu and comment on the differences. 

Write the analysis here.

#### Question3.	Pick the model that performs the best out of sample, and interpret its output by doing the following analysis of variable importance:
 
#### a.	First, for all stock characteristics, get variable importance by setting one predictor at a time to zero and finding the decrease in out of sample R2. Show a table of the 10 most important variables according to this measure, and give an economic interpretation. 

#### b.	Second, get a measure of the joint importance of all our “macro predictors” (i.e., those taken from Welch and Goyal 2008), by setting them all to zero and finding the decrease in out of sample R2. Comment on how important macroeconomic variables are relative to stock characteristics in predicting returns. 

#### c.	Repeat the two steps above, but by using a measure of the sensitivity of predictions to each input variable, as outlined in the lectures.


Write the analysis here.

#### Question4.	Fit a penalised linear model (LASSO) to the same data, using validation data to pick the best penalty (e.g., you can use the “sklearn” package in Python to do this easily). Compare its test data performance to the neural network. 

Write the analysis here.

#### Question5.	Suppose somebody tells you to collect 10 more micro or macro variables that can predict returns and are not in our current dataset. How would you choose those variables, based on the intuitions you have gained in this project?

Write the analysis here.